<a target="_blank" href="https://colab.research.google.com/github/AC-BO-Hackathon/Chihuahuas/blob/main/BO_batch_size.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Bayesian Optimization for `batch_size`

In [ ]:
!pip install spektral -q
!pip install scikit-optimize -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 2.0 MB/s eta 0:00:00


In [ ]:
#@title Enter the Size of the QM9 subset

amount = 1000 # @param {type:"integer"}

In [ ]:
"""
Optimización de hiperparámetros para un modelo GNN que predice una propiedad específica
del conjunto de datos QM9, utilizando Scikit-Optimize (skopt).
"""

import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from spektral.data import BatchLoader
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool, GraphMasking
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

################################################################################
# Configuración inicial
################################################################################
epochs = 10  # Número de épocas de entrenamiento
batch_size= 20

################################################################################
# Carga y preparación de datos
################################################################################
dataset = QM9(amount)  # Usar amount=None para todo el conjunto de datos

# Preparación de los datos para predecir solo la propiedad de índice 7
for graph in dataset:
    graph.y = graph.y[7].reshape(-1, 1)

# División entrenamiento/prueba
idxs = np.random.permutation(len(dataset))
split = int(0.9 * len(dataset))
idx_tr, idx_te = np.split(idxs, [split])
dataset_tr, dataset_te = dataset[idx_tr], dataset[idx_te]

################################################################################
# Definición del modelo
################################################################################
class Net(Model):
    def __init__(self):
        super(Net, self).__init__()
        self.masking = GraphMasking()
        self.conv1 = ECCConv(32, activation="relu")
        self.conv2 = ECCConv(32, activation="relu")
        self.global_pool = GlobalSumPool()
        self.dense = Dense(1)  # Salida para una propiedad

    def call(self, inputs):
        x, a, e = inputs
        x = self.masking(x)
        x = self.conv1([x, a, e])
        x = self.conv2([x, a, e])
        output = self.global_pool(x)
        output = self.dense(output)
        return output

################################################################################
# Espacio de búsqueda para la optimización de hiperparámetros
################################################################################
space = [
    Real(1e-3, 1e-1, name='learning_rate'),
    #Integer(16, 128, name='batch_size')
]

################################################################################
# Función objetivo para skopt
################################################################################
@use_named_args(space)
def objective(learning_rate):
    model = Net()
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse")

    loader_tr = BatchLoader(dataset_tr, batch_size=batch_size, mask=True)
    history = model.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs)

    loader_te = BatchLoader(dataset_te, batch_size=batch_size, mask=True)
    loss = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
    return loss

################################################################################
# Ejecución de la optimización Bayesiana
################################################################################
result = gp_minimize(objective, space, n_calls=20, random_state=0)




In [ ]:
################################################################################
# Resultados
################################################################################
print(f"Mejor valor de pérdida: {result.fun}")
print(f"Mejores hiperparámetros:\n- Learning Rate: {result.x[0]} ")